# Convolution Neural Networks: Light on Math Machine Learning

Here we implement a simple CNN that is trained and tested on MNIST dataset. We use Keras; a high level TensorFlow sublibrary to implement this.

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D,Dense,MaxPool2D,BatchNormalization,Flatten
from tensorflow.python.keras import backend as K

# Required for Data downaload and preparation
import struct
import gzip
import os
from six.moves.urllib.request import urlretrieve

c:\users\thushan\documents\python_virtualenvs\tensorflow_venv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Download Data and Preprocessing

Here we download the data (MNIST) dataset and perform some reshaping to make data `[28,28,1]` shape and normalize data into [0,1] range.

In [4]:
def maybe_download(url, filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename


def read_mnist(fname_img, fname_lbl, one_hot=False):
    print('\nReading files %s and %s'%(fname_img, fname_lbl))
    
    # Processing images
    with gzip.open(fname_img) as fimg:        
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        print(num,rows,cols)
        img = (np.frombuffer(fimg.read(num*rows*cols), dtype=np.uint8).reshape(num, rows, cols,1)).astype(np.float32)
        print('(Images) Returned a tensor of shape ',img.shape)
        
        img *= 1.0 / 255.0
    
    # Processing labels
    with gzip.open(fname_lbl) as flbl:
        # flbl.read(8) reads upto 8 bytes
        magic, num = struct.unpack(">II", flbl.read(8))               
        lbl = np.frombuffer(flbl.read(num), dtype=np.int8)
        if one_hot:
            one_hot_lbl = np.zeros(shape=(num,10),dtype=np.float32)
            one_hot_lbl[np.arange(num),lbl] = 1.0
        print('(Labels) Returned a tensor of shape: %s'%lbl.shape)
        print('Sample labels: ',lbl[:10])
    
    if not one_hot:
        return img, lbl
    else:
        return img, one_hot_lbl
    
    
# Download data if needed
url = 'http://yann.lecun.com/exdb/mnist/'
# training data
maybe_download(url,'train-images-idx3-ubyte.gz',9912422)
maybe_download(url,'train-labels-idx1-ubyte.gz',28881)
# testing data
maybe_download(url,'t10k-images-idx3-ubyte.gz',1648877)
maybe_download(url,'t10k-labels-idx1-ubyte.gz',4542)

# Read the training and testing data 
train_inputs, train_labels = read_mnist('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',True)
test_inputs, test_labels = read_mnist('t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz',True)

print('\nTrain size: ', train_inputs.shape[0])
print('\nTest size: ', test_inputs.shape[0])

Found and verified train-images-idx3-ubyte.gz
Found and verified train-labels-idx1-ubyte.gz
Found and verified t10k-images-idx3-ubyte.gz
Found and verified t10k-labels-idx1-ubyte.gz

Reading files train-images-idx3-ubyte.gz and train-labels-idx1-ubyte.gz
60000 28 28
(Images) Returned a tensor of shape  (60000, 28, 28, 1)
(Labels) Returned a tensor of shape: 60000
Sample labels:  [5 0 4 1 9 2 1 3 1 4]

Reading files t10k-images-idx3-ubyte.gz and t10k-labels-idx1-ubyte.gz
10000 28 28
(Images) Returned a tensor of shape  (10000, 28, 28, 1)
(Labels) Returned a tensor of shape: 10000
Sample labels:  [7 2 1 0 4 1 4 9 5 9]

Train size:  60000

Test size:  10000


# Defining the Model
Here we define the CNN, we first define set of convolution layers (`Conv2D`) and max-pooling layers (`MaxPool2D`) and then two fully-connected layers (`Dense`). You can see that the convolution and max-pooling layers are interleaved and the fully-connected layers sit at the end. Finally we compile the model, with the cross entropy loss and an optimizer, that will train your model. We will also monitor the accuracy during the training.

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config)
K.set_session(sess)

In [7]:
# Define a sequential model
model = Sequential()

# Added a convolution layer
model.add(Conv2D(32, (3,3), activation='relu', input_shape=[28, 28, 1]))

# Add a max pool lyer
model.add(MaxPool2D())

# Convolution layer
model.add(Conv2D(64, (3,3), activation='relu'))
# Max pool layer
model.add(MaxPool2D())

# More convolution, max pool
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## Running the Algorithm: Training and Testing

Running training and testing is quite easy, as long as we have the training/testing , input/output data loaded. It follows scikit-like syntax to implementn model fitting and evaulation.

In [8]:
n_epochs = 5 # Number of epochs the training runs for

batch_size = 100

x_train, y_train = train_inputs, train_labels
x_test, y_test = test_inputs, test_labels
    
for epoch in range(n_epochs):

    print('Training for epoch: ',epoch)    
    
    # Training for a single epoch (scikit-like syntax)
    model.fit(x_train, y_train, batch_size = batch_size)
    
    # Testing phase
    test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)  
    print('\tEpoch (', epoch ,') Test accuracy: ',test_acc[1])

Training for epoch:  0
Epoch 1/1
10000/10000 [==============================] - 1s 72us/step
	Epoch ( 0 ) Test accuracy:  0.9763000071048736
Training for epoch:  1
Epoch 1/1
10000/10000 [==============================] - 1s 72us/step
	Epoch ( 1 ) Test accuracy:  0.981300008893013
Training for epoch:  2
Epoch 1/1
10000/10000 [==============================] - 1s 75us/step
	Epoch ( 2 ) Test accuracy:  0.9854000067710876
Training for epoch:  3
Epoch 1/1
10000/10000 [==============================] - 1s 69us/step
	Epoch ( 3 ) Test accuracy:  0.9850000071525574
Training for epoch:  4
Epoch 1/1
10000/10000 [==============================] - 1s 70us/step
	Epoch ( 4 ) Test accuracy:  0.9860000085830688
